# **Gurobi 6: Análisis de Sensibilidad**


## **Introducción**  
En esta práctica, aprenderás a interpretar los resultados de un modelo de optimización mediante **análisis de sensibilidad**.  
Usaremos un **problema sencillo de producción** y obtendremos:  

📌 **Análisis de Sensibilidad en Gurobi:**  
✅ **Costos Reducidos** (Reduced Costs)  
✅ **Precios Duales** (Shadow Prices)  
✅ **Rango de Optimalidad** (Rango de los coeficientes de la función objetivo)  
✅ **Rango de Factibilidad** (Rango de las restricciones)  

---  
    


## **Descripción del Problema: Producción Óptima de Mesas y Sillas** 🏠  

Una empresa fabrica **Mesas y Sillas**.  
Cada producto requiere **madera y mano de obra**, pero estos recursos son limitados.  
El objetivo es **maximizar la utilidad** de la producción.  

📌 **Datos del problema:**  

- **Recursos disponibles:**  
  - Madera: **300 m²**  
  - Mano de obra: **200 horas**  

- **Requerimientos de producción (por unidad de producto):**  

| Producto | Madera (m²) | Mano de Obra (h) | Precio Venta (\$) | Costo Producción (\$) |
|----------|------------|------------------|----------------|----------------|
| Mesa     | 30         | 10               | 200            | 120            |
| Silla    | 20         | 15               | 150            | 80             |  

📌 **Función objetivo:**  
### **Función Objetivo:**
Maximizar la utilidad total:

$$
\max Z = \sum_{i=1}^{n} (P_i - C_i) x_i
$$

Donde:  
- $x_i $ es la cantidad de unidades a fabricar del producto $ i $.  
- $ P_i $ es el precio de venta del producto $ i $.  
- $ C_i $ es el costo de producción del producto $ i $.

---

### **Restricciones:**

#### **Disponibilidad de madera:**
$$
\sum_{i=1}^{n} a_{i1} x_i \leq 300
$$
Donde $a_{i1}$ es la cantidad de **madera** utilizada por unidad del producto $ i $.

#### **Disponibilidad de mano de obra:**
$$
\sum_{i=1}^{n} a_{i2} x_i \leq 200
$$
Donde $a_{i2}$ es la cantidad de **horas de mano de obra** requeridas por unidad del producto $ i $.

#### **No negatividad:**
$$
x_i \geq 0, \quad \forall i
$$
Donde $ x_i $representa la cantidad de unidades producidas y no puede ser negativa.  


    

## **Paso 1: Definir Parámetros del Modelo**

In [ ]:
# Descargar e instalar Gurobi desde la fuente oficial
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update
!apt-get install -y python3.10
!pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
!pip install gurobipy

# Importar la biblioteca
import gurobipy as gp
from gurobipy import GRB

print("Gurobi instalado y listo para usarse.")

In [ ]:
# Conjuntos
productos = ["Mesas", "Sillas"]

# Parámetros
costo_variable = {"Mesas": 120, "Sillas": 80}
precio_venta = {"Mesas": 200, "Sillas": 150}
recursos = ["Madera", "Mano de Obra"]
disponibilidad = {"Madera": 300, "Mano de Obra": 200}

# Requerimientos de cada producto
uso_recursos = {
    "Mesas": {"Madera": 30, "Mano de Obra": 10},
    "Sillas": {"Madera": 20, "Mano de Obra": 15}
}


## **Paso 2: Definir el Modelo de Optimización en Gurobi**

In [ ]:

# Crear el modelo
modelo = gp.Model("Produccion_Sensibilidad")

# Variables de decisión (cantidad de cada producto a fabricar)
x = modelo.addVars(productos, vtype=GRB.CONTINUOUS, lb=0, name="Produccion")

# Definir la función objetivo (maximizar utilidad)
modelo.setObjective(
    gp.quicksum((precio_venta[i] - costo_variable[i]) * x[i] for i in productos),
    GRB.MAXIMIZE
)

# Restricciones de recursos
for r in recursos:
    modelo.addConstr(
        gp.quicksum(uso_recursos[i][r] * x[i] for i in productos) <= disponibilidad[r],
        name=f"Restriccion_{r}"
    )

# Resolver el modelo
modelo.optimize()


## **Paso 3: Análisis de Sensibilidad**

In [ ]:

# Costos Reducidos (Reduced Costs)
print("\n📊 **Costos Reducidos**")
for v in modelo.getVars():
    print(f"{v.varName}: {v.RC:.2f}")

# Precios Duales (Shadow Prices)
print("\n📊 **Precios Duales (Shadow Prices)**")
for c in modelo.getConstrs():
    print(f"{c.ConstrName}: {c.Pi:.2f}")

# Rango de Optimalidad (Coeficientes de la función objetivo)
print("\n📊 **Rango de Optimalidad**")
for v in modelo.getVars():
    print(f"{v.varName}: {v.SAObjLow:.2f} <= coef <= {v.SAObjUp:.2f}")

# Rango de Factibilidad (Lado derecho de las restricciones)
print("\n📊 **Rango de Factibilidad**")
for c in modelo.getConstrs():
    print(f"{c.ConstrName}: {c.SARHSLow:.2f} <= RHS <= {c.SARHSUp:.2f}")



## **📌 Interpretación de los Resultados**  

✅ **Costos Reducidos:** Indican cuánto debería cambiar el coeficiente de la función objetivo para que la variable entre en la solución óptima.  
✅ **Precios Duales:** Muestran cuánto aumentaría el valor óptimo si se incrementara la disponibilidad del recurso en una unidad.  
✅ **Rango de Optimalidad:** Indica el intervalo dentro del cual un coeficiente de la función objetivo puede variar sin cambiar la solución óptima.  
✅ **Rango de Factibilidad:** Indica el intervalo dentro del cual un recurso disponible puede cambiar sin afectar la base óptima.  

---  

## **Ejercicios para el Alumno**  

1. **Modifica los costos de producción y analiza los cambios en los rangos de optimalidad.**  
2. **Aumenta la disponibilidad de madera en 50 unidades y analiza el nuevo precio dual.**  
3. **Cambia el coeficiente de la función objetivo de las Sillas y observa su impacto en los costos reducidos.**  
4. **Interpreta los resultados y explica en qué situaciones la empresa podría beneficiarse de cambios en los recursos o precios.**  

✍️ **Instrucciones:**  
- Modifica los parámetros en el código y vuelve a ejecutar el modelo.  
- Analiza los resultados e identifica los cambios en los valores de sensibilidad.  

🚀 ¡Ahora pon en práctica el análisis de sensibilidad con Gurobi!  
    